In [1]:
import numpy as np     #只需要下载numpy库即可
import random
import GridWorld_v2
import time
from IPython.display import clear_output

In [2]:


rows = 5      #记得行数和列数这里要同步改
columns = 5

# gridworld = GridWorld_v2.GridWorld_v2(rows=rows, columns=columns, forbiddenAreaNums=8, targetNums=2, seed = 52,forbiddenAreaScore=-10)
# gridworld = GridWorld_v2.GridWorld_v2(desc = [".#",".T"])             #赵老师4-1的例子
# gridworld = GridWorld_v2.GridWorld_v2(desc = ["##.T","...#","...."])  #随便弄的例子
gridworld = GridWorld_v2.GridWorld_v2(forbiddenAreaScore=-10, score=1,desc = [".....",".##..","..#..",".#T#.",".#..."]) 
#gridworld = GridWorld_v2(forbiddenAreaScore=-10, score=1,desc = ["T."]) 
gridworld.show()


value = np.zeros(rows*columns)       #初始化可以任意，也可以全0
qtable = np.zeros((rows*columns,5))  #初始化，这里主要是初始化维数，里面的内容会被覆盖所以无所谓


⬜️⬜️⬜️⬜️⬜️
⬜️🚫🚫⬜️⬜️
⬜️⬜️🚫⬜️⬜️
⬜️🚫✅🚫⬜️
⬜️🚫⬜️⬜️⬜️


In [12]:
def n_step_SARSA(gridworld:GridWorld_v2.GridWorld_v2,sarsaStep=5,epsilon=0.5,trajectorySteps=-1, learning_rate=0.001, final_epsilon=0.01, num_episodes=600, gamma = 0.9)->GridWorld_v2.GridWorld_v2:
    """
    这是n-step-SARSA算法

    Parameters:
    sarsaStep (int): 对应n-step-sarsa那个n的大小
    trajectorySteps (int): 寻路的轨迹长度，如果是-1，则为寻到目的则停止，否则参数即为trajectory长度
    learning_rate (float): 学习率，用于调节TD-target
    final_epsilon (float): epsilon-greedy的核心参数，0~1的浮点数，其中1则表示当前state所有决策概率一样，0则表示决策没有任何的随机性
    num_episodes (int): 表示模型迭代次数
    gamma (float): 近视远视程度

    Returns:
    GridWorld_v2.GridWorld_v2: 把模型返回回去
    """
    
    state_value = np.zeros((rows * columns))
    action_value = np.zeros((rows * columns, 5))
    policy = np.eye(5)[np.random.randint(0,5,size=(rows*columns))] 
    for episode in range(num_episodes):
        #清除输出，可以更好的展示策略
        time.sleep(0.2)
        clear_output(wait=True)
        
        print("episode",f"{episode}/{num_episodes}")
        if(epsilon > final_epsilon) :
            epsilon -= 0.001
        else:
            epsilon = final_epsilon

        # p1是目标方向的概率，p0是另外四个方向的概率
        p1 = 1-epsilon * (4/5)
        p0 = epsilon/5
        d = {1:p1, 0:p0}
        # policy_epsilon是policy取epsilon-greedy的概率决策
        print("p1",p1,"p0",p0)
        policy_epsilon = np.vectorize(d.get)(policy)

        #cnt数组用来检查每个state有多少次访问
        cnt = [0 for i in range(25)]
        
        initState=10
        initAction=random.randint(0,4)  

        if trajectorySteps==-1:
            stop_when_reach_target = True
        Trajectory = gridworld.getTrajectoryScore(nowState=initState, 
                                                  action=initAction, 
                                                  policy=policy_epsilon, 
                                                  steps=trajectorySteps, 
                                                  stop_when_reach_target=True)
        for i in range(sarsaStep):
            Trajectory.append((17,4,1,17,4)) #让他最后自转一下
        print("trajectorySteps",len(Trajectory))
        

        
        # 注意这里的返回值是大小为(trajectorySteps+1)的元组列表，因为把第一个动作也加入进去了
        
        steps = len(Trajectory)
        if steps>trajectorySteps:
            continue
        g = 1
        reward = 0
        for k in range(0, steps, 1):
            tmpstate, tmpaction, tmpscore, nextState, nextAction  = Trajectory[k]
            reward += tmpscore * g
            g *= gamma
        print("Trajectory_score", reward)


        
        for k in range(0, steps - sarsaStep + 1, 1):
            #State，Action，Reward，NextState，NextAction
            cnt[Trajectory[k][0]] += 1
            #n-step-SARSA 
            nowState,nowAction = -1,-1
            g = 1
            reward = 0
            for i in range(sarsaStep):
                #State，Action，Reward，NextState，NextAction
                tmpstate, tmpaction, tmpscore, nextState, nextAction  = Trajectory[k+i]
                if nowState == -1:
                    nowState, nowAction = tmpstate, tmpaction
                reward += tmpscore * g
                g *= gamma
            reward += g*action_value[nextState][nextAction]
            
            TD_error = action_value[nowState][nowAction] - reward
            action_value[nowState][nowAction] -= learning_rate * TD_error

        # for k in range(steps - sarsaStep+1, steps, 1):
        #     # print(k)
        #     #State，Action，Reward，NextState，NextAction
        #     tmpstate, tmpaction, tmpscore, nextState, nextAction  = Trajectory[k]
        #     cnt[tmpstate] += 1
        #     TD_error = action_value[tmpstate][tmpaction] - (tmpscore + gamma * action_value[nextState][nextAction])
        #     action_value[tmpstate][tmpaction] -= learning_rate * TD_error


        # policy improvement
        policy = np.eye(5)[np.argmax(action_value,axis=1)]  #qtable的最优值作为更新策略，并用独热码来表示
        policy_epsilon = np.vectorize(d.get)(policy)
    
        #输出每个state的访问次数
        print(np.array(cnt).reshape(5,5)) 

        state_value = np.sum(policy_epsilon * action_value,axis=1)
        mean_state_value = np.sum(policy_epsilon * action_value,axis=1).mean()
        
        gridworld.showPolicy(policy)
        print(np.round(action_value,decimals=3).reshape(-1,5))
        print(np.round(state_value,decimals=4).reshape(5,5))
        print("mean_state_value", mean_state_value)

    return gridworld
    

In [13]:
n_step_SARSA(gridworld,sarsaStep = 3, trajectorySteps=5000,final_epsilon=0.1,num_episodes = 400,gamma = 0.99)

episode 399/400
p1 0.9200000000000003 p0 0.019999999999999928
trajectorySteps 31
Trajectory_score 2.7739776150757676
[[ 1  1  1  1  1]
 [ 1  0  0  0  1]
 [15  0  0  0  1]
 [ 0  0  2  0  1]
 [ 0  0  1  1  1]]
➡️➡️➡️➡️⬇️
⬆️⏬⏩️⬆️⬇️
🔄⬅️⏬⬆️⬇️
⬆️⏩️✅⏪⬇️
🔄⏩️⬆️⬅️⬅️
[[-0.927 -0.736 -0.739 -0.869 -0.748]
 [-0.546 -0.419 -1.198 -0.467 -0.464]
 [-0.423 -0.18  -0.905 -0.205 -0.201]
 [-0.472 -0.355 -0.368 -0.36  -0.366]
 [-0.769 -0.646 -0.375 -0.39  -0.739]
 [-0.6   -1.989 -0.611 -1.17  -0.637]
 [-0.178 -0.346 -0.177 -0.201 -0.323]
 [-0.113 -0.104 -0.176 -0.182 -0.209]
 [-0.151 -0.16  -0.234 -1.026 -0.188]
 [-0.402 -0.523 -0.386 -0.528 -0.453]
 [-0.759 -0.951 -0.756 -1.001 -0.752]
 [-0.655 -0.504 -0.465 -0.317 -0.321]
 [-0.122 -0.061  0.051 -0.063 -0.266]
 [-0.15  -0.151 -0.379 -0.589 -0.163]
 [-0.341 -0.413 -0.332 -0.5   -0.339]
 [-0.594 -1.291 -0.596 -0.843 -0.602]
 [-0.072  0.252 -0.178 -0.046 -0.203]
 [-0.217 -0.112  0.071 -0.195  2.063]
 [-0.011 -0.028 -0.012  0.064 -0.068]
 [-0.167 -0.202 -0.14

In [94]:
n_step_SARSA(gridworld,sarsaStep = 2, trajectorySteps=5000)

episode 599/600
p1 0.992 p0 0.002
trajectorySteps 20001


In [114]:
g = 1
res = []
for i in range(15):
    g *= 0.9
    res.append(g*100//1/100)
print(res)

[0.9, 0.81, 0.72, 0.65, 0.59, 0.53, 0.47, 0.43, 0.38, 0.34, 0.31, 0.28, 0.25, 0.22, 0.2]


In [115]:
g = 1
res = []
for i in range(15):
    g *= 0.99
    res.append(g*100//1/100)
print(res)

[0.99, 0.98, 0.97, 0.96, 0.95, 0.94, 0.93, 0.92, 0.91, 0.9, 0.89, 0.88, 0.87, 0.86, 0.86]
